In [ ]:
import torch
import numpy as np
import random

from main.models import ConvNet, get_model_constructor
from main.utils import save_experiment
from dataclasses import dataclass
from main.active_learning import run_active_learning
from main.prepare_data import create_dataloaders
from configurations import get_config

%reload_ext autoreload
%autoreload 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# pick 10 super random numbers
fixed_seeds = [12345, 23456, 34567, 45678, 56789, 67890, 78901, 89012, 90123, 12340]

In [ ]:
config, experiment_name = get_config(
    method='entropy',
    min_samples=20,
    max_samples=200,
    acquisition_batch_size=10,
    dataset='repeated_mnist',
    )

print(f'Running {experiment_name} experiment')
save_results = True
num_runs = 10

In [ ]:
for i in range(num_runs):
    current_seed = fixed_seeds[i]
    torch.manual_seed(current_seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(current_seed)
        torch.cuda.manual_seed_all(current_seed)

    # load data
    train_loader, test_loader, pool_loader, active_learning_data = create_dataloaders(config)

    print(len(train_loader.dataset), len(test_loader.dataset), len(pool_loader.dataset))
    
    # get results
    results = run_active_learning(
        train_loader=train_loader,
        test_loader=test_loader, 
        pool_loader=pool_loader,
        active_learning_data=active_learning_data,
        model_constructor=get_model_constructor(config.al_method, config.dataset), 
        config=config, 
        device=device
        )

    # save results and configuration
    if save_results:
        experiment_id = experiment_name + '_' + str(i + 1)
        save_experiment(config, results, experiment_id, base_dir='')
    
    del train_loader, test_loader, pool_loader, active_learning_data